### First: Review Existing Unstructured Data and Diagram a New Structured Relational Data Model
<img src="/Users/yisheng/Desktop/Fetch_Analytics/diagram.png">

Above is the diagram and structure I have designed for the new relational data model.
Based on my research, below are my findings for the provided data sets:
* **brands.id** is NOT a subset of **receipts.id**, indicating that there is no direct one-to-one or subset relationship between these columns. This means the id in the Brands dataset does not strictly correspond to the id in the Receipts dataset. 
* **reward_receipts.receipt_id** is a subset of **receipts.id**, indicating a One-to-Many relationship. Each receipt can have multiple entries in the Reward Receipts dataset.
* **reward_receipts.barcode** is NOT a subset of **brands.barcode**, which means that some barcodes in Reward Receipts do not match the barcodes in the Brands dataset. This indicates an incomplete or optional relationship. However, by joining using the barcode column, I can get brand related attributes.

In [233]:
import json
import pandas as pd

# Load the unstructured JSON data
with open('/Users/yisheng/Desktop/Fetch_Analytics/users.json') as users:
    users_data = [json.loads(line) for line in users]

with open('/Users/yisheng/Desktop/Fetch_Analytics/brands.json') as brands:
    brands_data = [json.loads(line) for line in brands]

with open('/Users/yisheng/Desktop/Fetch_Analytics/receipts.json') as receipts:
    receipts_data = [json.loads(line) for line in receipts]

In [234]:
# Transform users.JSON data to flat structure
users = []
for user_record in users_data:
    users.append({
        'id': user_record['_id']['$oid'],
        'active': user_record.get('active', None),
        'created_date': pd.to_datetime(user_record['createdDate']['$date'], unit='ms') if 'createdDate' in user_record else None,
        'last_login_date': pd.to_datetime(record['lastLogin']['$date'], unit='ms') if 'lastLogin' in user_record else None,
        'role': user_record.get('role', None),
        'sign_up_source': user_record.get('signUpSource', None),
        'state': user_record.get('state', None)  # Handle missing 'state'
    })

# Transform brands.JSON data to flat structure, dimension table
brands = []
for brand_record in brands_data:
    brands.append({
        'id': brand_record['_id']['$oid'],
        'barcode': brand_record.get('barcode'),
        'category': brand_record.get('category'),
        'category_code': brand_record.get('categoryCode'),
        'cpg': brand_record['cpg']['$id']['$oid'],
        'ref': brand_record.get('ref'),
        'name': brand_record.get('name'),
        'top_brand': brand_record.get('topBrand')
    })

# Transform receipts.JSON to transactional data, fact table
# Separate receipts into two datasets since one receipt could possible contain multiple reward receipts item list
receipts = [] # To store main receipt data
reward_receipt_items = []  # To store reward receipt items, and added receipt_id from main receipt to link back to main receipt

for receipt_record in receipts_data:
    receipt_id = receipt_record['_id']['$oid']
    main_receipt = {
            'id': receipt_id,
            'bonus_points_earned': receipt_record.get('bonusPointsEarned'),
            'bonus_points_earned_reason': receipt_record.get('bonusPointsEarnedReason'),
            'create_date': pd.to_datetime(receipt_record['createDate']['$date'], unit = 'ms') if 'createDate' in receipt_record else None,
            'date_scanned': pd.to_datetime(receipt_record['dateScanned']['$date'], unit = 'ms') if 'dateScanned' in receipt_record else None,
            'finished_date': pd.to_datetime(receipt_record['finishedDate']['$date'], unit = 'ms') if 'finishedDate' in receipt_record else None,
            'modify_date': pd.to_datetime(receipt_record['modifyDate']['$date'], unit = 'ms') if 'modifyDate' in receipt_record else None,
            'points_awarded_date': pd.to_datetime(receipt_record['pointsAwardedDate']['$date'], unit = 'ms') if 'pointsAwardedDate' in receipt_record else None,
            'points_earned': float(receipt_record.get('pointsEarned', 0)),
            'purchase_date': pd.to_datetime(receipt_record['purchaseDate']['$date'], unit='ms') if 'purchaseDate' in receipt_record else None,
            'purchased_item_count': receipt_record.get('purchasedItemCount'),
            'rewards_receipt_status': receipt_record.get('rewardsReceiptStatus'),
            'total_spent': float(receipt_record.get('totalSpent', 0)),
            'user_id': receipt_record.get('userId')
    }
# Append main receipt data into receipts dataset
    receipts.append(main_receipt)

# Process reward receipt item list data if exists
    if 'rewardsReceiptItemList' in receipt_record:
        for item in receipt_record['rewardsReceiptItemList']:
            # Extracting item-specific fields
            reward_item = {
                'receipt_id': receipt_id,  # Reference to the main receipt
                'barcode': item.get('barcode'),
                'description': item.get('description'),
                'discounted_item_price': float(item.get('discountedItemPrice', 0)),
                'final_price': float(item.get('finalPrice', 0)),
                'item_number': item.get('itemNumber'),
                'item_price': float(item.get('itemPrice', 0)),
                'needs_fetch_review': item.get('needsFetchReview', False),
                'needs_fetch_review_reason': item.get('needsFetchReviewReason'),
                'original_meta_brite_quantity_purchased': item.get('originalMetaBriteQuantityPurchased'),
                'partner_item_id': item.get('partnerItemId'),
                'points_earned': float(item.get('pointsEarned', 0)),
                'points_payer_id': item.get('pointsPayerId'),
                'quantity_purchased': item.get('quantityPurchased'),
                'rewards_group': item.get('rewardsGroup'),
                'rewards_product_partner_id': item.get('rewardsProductPartnerId'),
                'target_price': float(item.get('targetPrice', 0)),
                'user_flagged_barcode': item.get('userFlaggedBarcode'),
                'user_flagged_new_item': item.get('userFlaggedNewItem', False),
                'user_flagged_price': float(item.get('userFlaggedPrice', 0)),
                'user_flagged_quantity': item.get('userFlaggedQuantity')
            }
            reward_receipt_items.append(reward_item)


# Convert the structured data into a Pandas DataFrame
users_df = pd.DataFrame(users)
users_df.head()

brands_df = pd.DataFrame(brands)
brands_df.head()

receipts_df = pd.DataFrame(receipts)
receipts_df.head()

reward_receipts_df = pd.DataFrame(reward_receipt_items)
reward_receipts_df.head()

,receipt_id,barcode,description,discounted_item_price,final_price,item_number,item_price,needs_fetch_review,needs_fetch_review_reason,original_meta_brite_quantity_purchased,...,points_earned,points_payer_id,quantity_purchased,rewards_group,rewards_product_partner_id,target_price,user_flagged_barcode,user_flagged_new_item,user_flagged_price,user_flagged_quantity
0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,0.0,26.0,None,26.0,False,None,NaN,...,0.0,None,5.0,None,None,0.0,4011,True,26.0,5.0
1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,0.0,1.0,None,1.0,False,None,NaN,...,0.0,None,1.0,None,None,0.0,None,False,0.0,NaN
2,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,0.0,10.0,None,10.0,True,USER_FLAGGED,NaN,...,0.0,5332f5fbe4b03c9a25efd0ba,1.0,DORITOS SPICY SWEET CHILI SINGLE SERVE,5332f5fbe4b03c9a25efd0ba,0.0,028400642255,True,10.0,1.0
3,5ff1e1f10a720f052300057a,None,None,0.0,0.0,None,0.0,False,None,NaN,...,0.0,None,NaN,None,None,0.0,4011,True,26.0,3.0
4,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,0.0,28.0,None,28.0,False,None,NaN,...,0.0,None,4.0,None,None,0.0,4011,True,28.0,4.0


In [235]:
# Download as csv to better view 3 datasets
users_df.to_csv('users.csv', index = False)
brands_df.to_csv('brands.csv', index = False)
receipts_df.to_csv('receipts.csv', index = False)
reward_receipts_df.to_csv('reward_receipts.csv', index = False)

### Second: Write queries that directly answer predetermined questions from a business stakeholder

Each question will be answered within corresponding code block.
* What are the top 5 brands by receipts scanned for most recent month?
* How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?
* When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
* When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
* Which brand has the most spend among users who were created within the past 6 months?
* Which brand has the most transactions among users who were created within the past 6 months?

In [268]:
from sqlalchemy import create_engine

# Create database using SQLAlchemy
engine = create_engine('sqlite:///:memory:')

# Write DataFrames to the database
users_df.to_sql('users', engine, index = False, if_exists = 'replace')
brands_df.to_sql('brands', engine, index = False, if_exists = 'replace')
receipts_df.to_sql('receipts', engine, index = False, if_exists = 'replace')
reward_receipts_df.to_sql('reward_receipts', engine, index = False, if_exists = 'replace')


6941

In [269]:
# 1. What are the top 5 brands by receipts scanned for most recent month?
querytest = """
with most_recent_month as (
    select strftime('%Y-%m', max(date_scanned)) as recent_month
    from receipts
)
select * from most_recent_month;
"""
test = pd.read_sql_query(querytest, engine)
print(test)

  recent_month
0      2021-03


In [270]:
# 1. What are the top 5 brands by receipts scanned for most recent month?
# Note: If I understand the question correctly, it is asking top 5 brands by receipts scanned for most recent month where 
# most recent month would be 2021-03 which it can be verified from most_recent_month CTE.
# However, by leverging this CTE to further filter data to get the final result of 2021-03 it returns no data, indicating no
# reward receipts is in 2021-03. Therefore, I commented out the where condition to still show the top 5 brands by most 
# recent month apart from 2021-03, which is 2021-01.
query1 = """
with most_recent_month as (
    select strftime('%Y-%m', max(date_scanned)) as recent_month
            , strftime('%Y-%m', date(max(date_scanned), '-1 month')) as previous_month
    from receipts
)

, recent_month_data as(
    select
        b.name as brand_name
        , count(distinct r.id) as receipts_scanned
    from receipts r
        left join reward_receipts rr 
            on r.id = rr.receipt_id
        join brands b 
            on rr.barcode = b.barcode
    --where strftime('%Y-%m', r.date_scanned) = (select recent_month from most_recent_month)
    group by 1
    order by 2 desc
)
, previous_month_data as(
    select
        b.name as brand_name
        , count(distinct r.id) as receipts_scanned
    from receipts r
        left join reward_receipts rr 
            on r.id = rr.receipt_id
        join brands b 
            on rr.barcode = b.barcode
    --where strftime('%Y-%m', r.date_scanned) = (select previous_month from most_recent_month)
    group by 1
    order by 2 desc
)

SELECT 
    COALESCE(rmd.brand_name, pmd.brand_name) AS brand_name,
    COALESCE(rmd.receipts_scanned, 0) AS recent_month_receipts,
    COALESCE(pmd.receipts_scanned, 0) AS previous_month_receipts
FROM recent_month_data rmd
FULL OUTER JOIN previous_month_data pmd
    ON rmd.brand_name = pmd.brand_name
ORDER BY recent_month_receipts DESC, previous_month_receipts DESC
LIMIT 5;

"""
question1 = pd.read_sql_query(query1, engine)
print(question1)

              brand_name  recent_month_receipts  previous_month_receipts
0               Tostitos                     11                       11
1                Swanson                     11                       11
2  Cracker Barrel Cheese                     10                       10
3                  Prego                      4                        4
4        Diet Chris Cola                      4                        4


In [271]:
# Compare top 5 brands by receipts scanned for the recent month vs. the previous month
# Q2 is based on the assumption from Q1 as well, where I believe the logic I am writing in SQL is correctly capturing
# most recent month (N month) and (N-1 month) accordingly
query2 = """
with most_recent_month as (
    select strftime('%Y-%m', max(date_scanned)) as recent_month
            , strftime('%Y-%m', date(max(date_scanned), '-1 month')) as previous_month
    from receipts
)

, recent_month_data as(
    select
        b.name as brand_name
        , count(distinct r.id) as receipts_scanned
    from receipts r
        left join reward_receipts rr 
            on r.id = rr.receipt_id
        join brands b 
            on rr.barcode = b.barcode
    --where strftime('%Y-%m', r.date_scanned) = (select recent_month from most_recent_month)
    group by 1
    order by 2 desc
)
, previous_month_data as(
    select
        b.name as brand_name
        , count(distinct r.id) as receipts_scanned
    from receipts r
        left join reward_receipts rr 
            on r.id = rr.receipt_id
        join brands b 
            on rr.barcode = b.barcode
    --where strftime('%Y-%m', r.date_scanned) = (select previous_month from most_recent_month)
    group by 1
    order by 2 desc
)

SELECT 
    COALESCE(rmd.brand_name, pmd.brand_name) AS brand_name,
    COALESCE(rmd.receipts_scanned, 0) AS recent_month_receipts,
    COALESCE(pmd.receipts_scanned, 0) AS previous_month_receipts
FROM recent_month_data rmd
FULL OUTER JOIN previous_month_data pmd
    ON rmd.brand_name = pmd.brand_name
ORDER BY recent_month_receipts DESC, previous_month_receipts DESC
LIMIT 5;

"""
question2 = pd.read_sql_query(query2, engine)
print(question2)


              brand_name  recent_month_receipts  previous_month_receipts
0               Tostitos                     11                       11
1                Swanson                     11                       11
2  Cracker Barrel Cheese                     10                       10
3                  Prego                      4                        4
4        Diet Chris Cola                      4                        4


In [264]:
# 3. Compare average spend for receipts with 'Accepted' vs. 'Rejected'
# Note: I checked original receipt.json file, there's no such value of Accepted / ACCEPTED in rewards_receipt_status column
# I assumed the question is referring to status "Finsihed"
query2 = """
select 
    rewards_receipt_status
    , avg(total_spent) as average_spend
from receipts
where rewards_receipt_status in ('FINISHED', 'REJECTED')
group by 1;
"""
question3 = pd.read_sql_query(query2, engine)
print(question3)

  rewards_receipt_status  average_spend
0               FINISHED      80.854305
1               REJECTED      23.326056


In [265]:
# 4. Compare total items purchased for receipts with 'Accepted' vs. 'Rejected'
# Note: Same as Q3, I assumed the question is referring to status "Finsihed" instead of "Accepted"
query4 = """
select 
    rewards_receipt_status,
    sum(purchased_item_count) as total_items_purchased
from receipts
where rewards_receipt_status in ('FINISHED', 'REJECTED')
group by 1;
"""
question4 = pd.read_sql_query(query3, engine)
print(question4)

  rewards_receipt_status  total_items_purchased
0               FINISHED                 8184.0
1               REJECTED                  173.0


In [266]:
# 5. Brand with the most spend among users created in the past 6 months
query5 = """
with users_last_6_months AS (
    select id as user_id
    from users
    where created_date >= date((select max(created_date) from users), '-6 months')
),
brand_spend as (
    select 
        b.name as brand_name,
        sum(r.total_spent) as total_spend
    from receipts r
        join reward_receipts rr 
            on r.id = rr.receipt_id
        join brands b 
            on rr.barcode = b.barcode
    where r.user_id in (select user_id from users_last_6_months)
    group by 1
)
select brand_name
        , total_spend
from brand_spend
order by 2 desc
limit 1;
"""
question5 = pd.read_sql_query(query5, engine)
print(question5)

  brand_name  total_spend
0   Tostitos     15799.37


In [267]:
# 6. Which brand has the most transactions among users created within the past 6 months?
query6 = """
with users_last_6_months AS (
    select id as user_id
    from users
    where created_date >= date((select max(created_date) from users), '-6 months')
),
brand_transactions as (
    select 
        b.name as brand_name,
        count(distinct r.id) as transactions_count
    from receipts r
        join reward_receipts rr 
            on r.id = rr.receipt_id
        join brands b 
            on rr.barcode = b.barcode
    where r.user_id in (select user_id from users_last_6_months)
    group by 1
)
select brand_name
        , transactions_count
from brand_transactions
order by 2 desc
limit 1;
"""
question6 = pd.read_sql_query(query6, engine)
print(question6)

  brand_name  transactions_count
0    Swanson                  11


### Third: Evaluate Data Quality Issues in the Data Provided

In [244]:
# First look at the summary for each data set
# List of DataFrames and their names
datasets = {
    "users": users_df,
    "brands": brands_df,
    "receipts": receipts_df,
    "reward_receipts": reward_receipts_df
}

# Iterate and display summaries
for name, df in datasets.items():
    print(f"Summary of {name} DataFrame:")
    print("Data Types and Non-Null Counts:")
    print(df.info())  # Data types and non-null counts
    print("\nStatistical Summary:")
    print(df.describe(include='all'))  # Statistical summary including all columns
    print("\nSample Rows:")
    print(df.head())  # Display the first few rows for context
    print("\n" + "="*50 + "\n")


Summary of users DataFrame:
Data Types and Non-Null Counts:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               495 non-null    object        
 1   active           495 non-null    bool          
 2   created_date     495 non-null    datetime64[ns]
 3   last_login_date  433 non-null    datetime64[ns]
 4   role             495 non-null    object        
 5   sign_up_source   447 non-null    object        
 6   state            439 non-null    object        
dtypes: bool(1), datetime64[ns](2), object(4)
memory usage: 23.8+ KB
None

Statistical Summary:
                              id active                   created_date  \
count                        495    495                            495   
unique                       212      2                            NaN   
top     54943462e4b07e684157a532   True  

#### When conducting analysis, I observed several issues:
1. Null or Missing Values:

* Columns like **category_code**, **ref**, and **top_brand** in *brands* have many missing values.
* **bonus_points_earned**, **points_awarded_date**, **purchased_item_count**, and **purchase_date** in *receipts* have missing values.
* Many columns in *reward_receipts*, such as **item_number**, **original_meta_brite_quantity_purchased**, and **points_payer_id**, have large portion of missing data.
* In *users*, columns like **last_login_date**, **sign_up_source**, and **state** have null values.

2. Data Consistency:

* In *receipts*, mismatched or unrealistic dates (e.g., modify_date earlier than create_date or purchase_date in the future).
* In *reward_receipts*, inconsistencies between discounted_item_price, final_price, and item_price. 

3. Unusual Data:

* Negative or zero values in numeric fields like *points_earned*, *total_spent*, or *quantity_purchased*, which doesn't make sense. For example, if it's refund transaction, I would expect another column specifically storing refunds value other than putting negative value in *total_spent*.
* Null category_code in brands for valid category.

4. Foreign Key Issue:
* Broken relationships between tables (e.g., user_id in *receipts* not existing in *users*). (I assumed all user need to have either user_id (membership_id) to use the service)

In [252]:
# Define a dictionary of DataFrames
data = {
    "brands": brands_df,
    "receipts": receipts_df,
    "reward_receipts": reward_receipts_df,
    "users": users_df,
}

# Create an in-memory SQLite database and load data
# conn = sqlite3.connect(":memory:")
from sqlalchemy import create_engine

# Create database using SQLAlchemy
engine = create_engine('sqlite:///:memory:')

# Load each DataFrame into the SQLite database
for table_name, df in data.items():
    df.to_sql(table_name, engine, index=False, if_exists="replace")

# Queries to identify data quality issues
queries = {
    "missing_category_code": """
        select count(*) as missing_category_code_count
        from brands
        where category_code is null;
    """,
    "missing_receipt_dates": """
        select count(*) as missing_purchase_date_count
        from receipts
        where purchase_date is null;
    """,
    "negative_or_zero_values": """
        select count(*) as invalid_quantity_purchased
        from reward_receipts
        where quantity_purchased <= 0 or points_earned < 0;
    """,
    "broken_user_relationships": """
        select count(*) as invalid_user_ids
        from receipts
        where user_id not in (select id from users group by 1);
    """,
    "inconsistent_dates": """
        select count(*) as inconsistent_dates_count
        from receipts
        where modify_date < create_date or purchase_date > date('now');
    """,
    "missing_brand_attributes": """
        select 
            rr.barcode
            , name
        from reward_receipts rr
            join brands b
                on rr.barcode = b.barcode
        group by 1,2
        order by 1 desc
    """,
}

# Execute the queries and collect results
results = {name: pd.read_sql_query(query, engine) for name, query in queries.items()}

# Display results
results

{'missing_category_code':    missing_category_code_count
 0                          650,
 'missing_receipt_dates':    missing_purchase_date_count
 0                          448,
 'negative_or_zero_values':    invalid_quantity_purchased
 0                           0,
 'broken_user_relationships':    invalid_user_ids
 0               148,
 'inconsistent_dates':    inconsistent_dates_count
 0                         0,
 'missing_brand_attributes':          barcode                   name
 0   511111904175        Pepperidge Farm
 1   511111902690                  Kraft
 2   511111901587            Rice A Roni
 3   511111802358  Cracker Barrel Cheese
 4   511111704140        Diet Chris Cola
 5   511111704140                  Prego
 6   511111602118                 Jell-O
 7   511111518044       Sargento® Cheese
 8   511111502142            Grey Poupon
 9   511111204206                Swanson
 10  511111104537           Kettle Brand
 11  511111104186          Pacific Foods
 12  51111110145

### Fourth: Communicate with Stakeholders
Hi Product/Business Leader,

I would like to share a summary of our recent data analysis, outline key findings, and next steps to improve data quality and scalability.

- Key Findings
   - Missing Data:
      - 650 brands records lack category_code
      - 448 receipts records lack purchase_date
   - Broken Relationships:
      - 148 receipts reference invalid user_ids  

- Open Questions
   - Should missing category_code or purchase_date values be inferred or flagged as errors?
   - What action should be taken for receipts with invalid user_ids?

- Next Steps
   - Align with stakeholders on data validation rules (e.g., mandatory fields, handling of invalid references, whether to replace null as 0 in certain cases)
   - Trace data lineage to identify the root cause of issues

- Scalability Considerations
   - Performance: Will optimize queries with indexing and validation at ETL stages once we have data quality issues solved
   - Growth: Plan for partitioning and database scaling as data volume increases in the future
Please let me know if you’d like to discuss this further or have additional insights.

Best Regards,
Sean